In [32]:
import numpy as np
import vtk
from vtk.util import numpy_support
import os

In [4]:
#get reader to load tiff file
reader = vtk.vtkTIFFReader()
reader.SetFileName(r'C:\Users\miao1\Data\focused-fast-scans\trimmed_stacks\Trimmed_cell3_Iter_0_ch0_stack0000_3nm_0000000msec_0009286310msecAbs.tif')
reader.Update()

#extent = reader.GetDataExtent()
#dims = [extent[1]-extent[0]+1, extent[3]-extent[2]+1, extent[5]-extent[4]+1]
#print('dims', dims)

#shape array into dims of volume (currently not used)
#pointData = reader.GetOutput().GetPointData()
#arrayData = pointData.GetArray(0)
#vol = numpy_support.vtk_to_numpy(arrayData)
#vol = vol.reshape(dims, order='F')

dims [259, 279, 289]


In [125]:
%%time
#use a threshold to create a binary mask
th = 600
thf = vtk.vtkImageThreshold()
thf.SetInputConnection(reader.GetOutputPort())
thf.ThresholdByLower(th)
thf.ReplaceInOn()
thf.SetInValue(0)  # values below th -> 0
thf.ReplaceOutOn()
thf.SetOutValue(1)  # values above th -> 0
thf.Update()

Wall time: 8.01 ms


In [126]:
#smooth the mask. sub 4 voxel is likely noise

c = vtk.vtkImageOpenClose3D()
c.SetInputConnection(thf.GetOutputPort())
c.SetOpenValue(0)
c.SetCloseValue(1)
c.SetKernelSize(3,3,3)
c.Update()

o = vtk.vtkImageOpenClose3D()
o.SetInputConnection(c.GetOutputPort())
o.SetOpenValue(1)
o.SetCloseValue(0)
o.SetKernelSize(3,3,3)
o.Update()

In [127]:
#extract surface using marching cubes
mc = vtk.vtkDiscreteMarchingCubes()
mc.SetInputConnection(o.GetOutputPort())
mc.GenerateValues(1, 1, 1)
mc.Update()

In [128]:
writer = vtk.vtkTIFFWriter()
writer.SetInputConnection(o.GetOutputPort())
writer.SetFileName("cell_th.tif")
writer.Write()

In [131]:
#We only want the largest connected component
con = vtk.vtkPolyDataConnectivityFilter()
con.SetInputData(mc.GetOutput())
con.SetExtractionModeToLargestRegion()
con.Update()

In [153]:
%%time
#use laplacian filter to smooth surface
s = 10
r = 0.1
smooth = vtk.vtkSmoothPolyDataFilter()

if(gSmoothingSteps > 0):
    print("smoothing...")
    smooth.SetNumberOfIterations(s)
    smooth.SetInputData(con.GetOutput())
    smooth.SetRelaxationFactor(r)
    smooth.Update()

smoothing...
Wall time: 144 ms


In [154]:
#save smoothed surface as stl
writer = vtk.vtkSTLWriter()
writer.SetInputConnection(smooth.GetOutputPort())
writer.SetFileTypeToBinary()
out = "cell_{}_{}.stl".format(th,s)
writer.SetFileName(out)
writer.Write()

1

In [155]:
properties = vtk.vtkMassProperties()
properties.SetInputConnection(smooth.GetOutputPort())
print("surface area", properties.GetSurfaceArea())
print("volume", properties.GetVolume())

surface area 57145.166582487866
volume 1025830.9611052148


In [156]:
pca = vtk.vtkPCAAnalysisFilter()
pca.SetInputConnection(smooth.GetOutputPort())
pca.Update()